# The Amazing Orbital Intelligence Pony Identifier
This is a project to try and get a grip on how pytorch/torch works. Here's hoping for the best!

First, some basic imports. Specifically the `array` type from numpy, since that is a convenient way to represent the data we'll work with.

In [13]:
from numpy import array
import numpy.typing as npt
from typing import Literal
from math import floor

Next, setup. We're borrowing some pre-set values from [an existing repository](https://github.com/Lightning-AI/dl-fundamentals/blob/main/unit01-ml-intro/exercises/solutions/unit01_excercise_1/solution_ex1_early-stop.ipynb) here.



In [14]:
dataset = array([[ 0.77, -1.14],
       [-0.33,  1.44],
       [ 0.91, -3.07],
       [-0.37, -1.91],
       [-0.63, -1.53],
       [ 0.39, -1.99],
       [-0.49, -2.74],
       [-0.68, -1.52],
       [-0.1 , -3.43],
       [-0.05, -1.95],
       [ 3.88,  0.65],
       [ 0.73,  2.97],
       [ 0.83,  3.94],
       [ 1.59,  1.25],
       [ 1.14,  3.91],
       [ 1.73,  2.8 ],
       [ 1.31,  1.85],
       [ 1.56,  3.85],
       [ 1.23,  2.54],
       [ 1.33,  2.03]])

true_values = array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

class BasicPerceptron:
    def __init__(self, w_count:int):
        self.number_of_weigths:int = w_count
        self.weights:list[float] = [0.0] * w_count
        self.bias:float = 0.0
    
    def update(self,data:tuple[float,float],label:Literal[1,0]) -> float:
        assert len(data) == len(self.weights)

        model_result:float = self.forward(data)
        
        error:float = float(label) - model_result
        self.bias += error

        for index in range(len(self.weights)):
            self.weights[index] += (error * data[index]) #??
        
        return error
    
    def forward(self, input_data:array) -> Literal[1,0]:
        assert len(input_data) == len(self.weights)

        weighted_sum = self.bias
        for weight, in_val in zip(self.weights,input_data):
            weighted_sum += weight * in_val
        if weighted_sum > 0:
            return 1
        return 0
    
def init_perceptron(epochs:int, w_count:int, update_weights:array, update_labels:array) -> BasicPerceptron:
    assert len(update_weights) == len(update_labels)
    assert all(len(data) == w_count for data in update_weights)
    perceptron = BasicPerceptron(w_count)

    for epoch in range(epochs):
        absolute_err:float = 0.0
        for data,label in zip(update_weights,update_labels):
            absolute_err += abs(perceptron.update(data,label))
        print(f"Epoch {epoch+1}: e {floor(absolute_err)}")
        if not bool(absolute_err):
            print("Error is zero, ending now.")
            break

    return perceptron



Now that everything is set up and initialized, run the actual perceptron.


In [17]:
my_perceptron = init_perceptron(5,2,dataset,true_values)

x = [1.1,2.1]
print(f"{x} -> {my_perceptron.forward(x)}")

Epoch 1: e 1
Epoch 2: e 3
Epoch 3: e 1
Epoch 4: e 0
Error is zero, ending now.
[1.1, 2.1] -> 1
